In [1]:
import pandas as pd 
from utils import extract_colors_image,extract_colors_url

In [3]:
url=extract_colors_url("https://nestorevintage.fr/cdn/shop/files/307FDA5D-D3FB-48D6-8DA2-2478DF104BD3.jpg?v=1738348432&width=2048")

c:\Users\BahaARFAOUI\OneDrive - CBTW\Bureau\colorfact\colorfact\utils.py:80: RuntimeWarning: overflow encountered in scalar subtract
  a_std = float(a_opencv - 128)


In [4]:
url.colors

[[58.03921568627452, 251.0, 26.0], [46.27450980392157, 253.0, 23.0]]

In [4]:
image=extract_colors_image(image_path="data/doudoune.png")

In [5]:
image.colors

[[40.3921568627451, -7.0, 6.0], [23.52941176470588, -7.0, 6.0]]

In [6]:
im=image.remove_background_grabcut()

In [7]:
from PIL import Image
img = Image.open(im)

AttributeError: 'numpy.ndarray' object has no attribute 'read'

In [1]:
from utils import LoadToolsConfig

In [2]:
TOOLS_CFG = LoadToolsConfig()

In [3]:
TOOLS_CFG.faiss_index

'colorfact/vector_store/faiss_index.idx'